In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install pyspark

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:15 http://ppa.lau

In [ ]:
from pyspark import SparkContext

sc = SparkContext.getOrCreate()

sayilar = [1,2,3,4,5,6,7,8]
kareler = []
for sayi in sayilar:
  kareler.append(sayi*sayi)
print(sayilar)
print(kareler)

sayilarRDD = sc.parallelize(sayilar)
#Transformation (map,filter,flatmap,sort,....vs)
#Action
print("Aşağıdakiler Paralel yapılan işlem sonucu")
karelerRDD = sayilarRDD.map(lambda s : s*s)
print(karelerRDD.collect())

irisRDD= sc.textFile("/content/iris-dataset.txt")
print(irisRDD.collect())

[1, 2, 3, 4, 5, 6, 7, 8]
[1, 4, 9, 16, 25, 36, 49, 64]
Aşağıdakiler Paralel yapılan işlem sonucu
[1, 4, 9, 16, 25, 36, 49, 64]
['sepal-length,sepal-width,petal-length,petal-width,class', '5.1,3.5,1.4,0.2,Iris-setosa', '4.9,3.0,1.4,0.2,Iris-setosa', '4.7,3.2,1.3,0.2,Iris-setosa', '4.6,3.1,1.5,0.2,Iris-setosa', '5.0,3.6,1.4,0.2,Iris-setosa', '5.4,3.9,1.7,0.4,Iris-setosa', '4.6,3.4,1.4,0.3,Iris-setosa', '5.0,3.4,1.5,0.2,Iris-setosa', '4.4,2.9,1.4,0.2,Iris-setosa', '4.9,3.1,1.5,0.1,Iris-setosa', '5.4,3.7,1.5,0.2,Iris-setosa', '4.8,3.4,1.6,0.2,Iris-setosa', '4.8,3.0,1.4,0.1,Iris-setosa', '4.3,3.0,1.1,0.1,Iris-setosa', '5.8,4.0,1.2,0.2,Iris-setosa', '5.7,4.4,1.5,0.4,Iris-setosa', '5.4,3.9,1.3,0.4,Iris-setosa', '5.1,3.5,1.4,0.3,Iris-setosa', '5.7,3.8,1.7,0.3,Iris-setosa', '5.1,3.8,1.5,0.3,Iris-setosa', '5.4,3.4,1.7,0.2,Iris-setosa', '5.1,3.7,1.5,0.4,Iris-setosa', '4.6,3.6,1.0,0.2,Iris-setosa', '5.1,3.3,1.7,0.5,Iris-setosa', '4.8,3.4,1.9,0.2,Iris-setosa', '5.0,3.0,1.6,0.2,Iris-setosa', '5.0,3.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler,StringIndexer
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator

spark = SparkSession.builder.appName("Spark ML").getOrCreate()
irisDF = spark.read.option("inferSchema","true").option("header","true").csv("/content/iris-dataset.txt")
#irisDF.show()
vec = VectorAssembler(inputCols=["sepal-length","sepal-width","petal-length","petal-width"],
                      outputCol="features")
irisDF = vec.transform(irisDF)
indexer = StringIndexer(inputCol="class",outputCol="label")
irisDF = indexer.fit(irisDF).transform(irisDF)
irisDF = irisDF.select("features","label")
#irisDF.show()
classifier= DecisionTreeClassifier()
#makine öğrenmesi yapılıyor ( Model Training)
#Model başarısını görmek için
trainDF, testDF = irisDF.randomSplit([0.75,0.25],seed=1234)

model = classifier.fit(trainDF)
print("Makine öğrenmesi tamamlandı")
#print(model.toDebugString)

resultDF = model.transform(testDF)
resultDF.show(100)
eva = MulticlassClassificationEvaluator(metricName="accuracy")
basari = eva.evaluate(resultDF)
print("Başarı oranımız : ",basari)

Makine öğrenmesi tamamlandı
+-----------------+-----+--------------+-------------+----------+
|         features|label| rawPrediction|  probability|prediction|
+-----------------+-----+--------------+-------------+----------+
|[4.4,2.9,1.4,0.2]|  0.0|[34.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|[4.5,2.3,1.3,0.3]|  0.0|[34.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|[4.9,3.1,1.5,0.1]|  0.0|[34.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|[5.0,3.0,1.6,0.2]|  0.0|[34.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|[5.0,3.2,1.2,0.2]|  0.0|[34.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|[5.0,3.3,1.4,0.2]|  0.0|[34.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|[5.0,3.4,1.5,0.2]|  0.0|[34.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|[5.0,3.5,1.3,0.3]|  0.0|[34.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|[5.0,3.6,1.4,0.2]|  0.0|[34.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|[5.1,2.5,3.0,1.1]|  1.0|[0.0,32.0,0.0]|[0.0,1.0,0.0]|       1.0|
|[5.1,3.5,1.4,0.3]|  0.0|[34.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|[5.3,3.7,1.5,0.2]|  0.0|[34.0,0.0,0.0]|[1.0,0.0

In [ ]:
yeniDF = spark.read.option("inferSchema","true").option("header","true").csv("/content/yeni-veriler.txt")
yeniDF.show()
yeniDF = vec.transform(yeniDF)
#Tahmin yapıyorum ( Prediction)
sonucDF = model.transform(yeniDF)
print("bitti gitti")
sonucDF.show()

+------------+-----------+------------+-----------+
|sepal-length|sepal-width|petal-length|petal-width|
+------------+-----------+------------+-----------+
|         4.6|        2.2|         1.4|        0.2|
|         6.0|        3.3|         1.4|        0.2|
|         7.0|        1.2|         4.7|        1.4|
|         6.4|        3.2|         4.5|        1.5|
|         6.2|        4.4|         5.4|        2.3|
|         5.9|        3.0|         5.1|        1.9|
+------------+-----------+------------+-----------+

bitti gitti
+------------+-----------+------------+-----------+-----------------+--------------+-------------+----------+
|sepal-length|sepal-width|petal-length|petal-width|         features| rawPrediction|  probability|prediction|
+------------+-----------+------------+-----------+-----------------+--------------+-------------+----------+
|         4.6|        2.2|         1.4|        0.2|[4.6,2.2,1.4,0.2]|[34.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|         6.0|        3.3| 